# [문장의 유사도를 N-gram으로 분석하기]

## 1. 레벤슈타인 거리(Lvenshtein distance)
두 개의 문자열이 어느 정도 다른지를 나타내는 것. 편집거리(Edit Distance)라고도 한다.  
철자 오류 수정, 비슷한 어구 검색 등에 사용되고 있다. 또한 의학 분야에서는 DNA 배열의 유사성을 판단할 때도 사용하고 있다.  


### 1) 파이썬으로 레벤슈타인 거리를 계산하는 프로그램

In [1]:
def calc_distance(a, b):
    ''' 레벤슈타인 거리 계산하기 '''
    if a == b: return 0
    a_len = len(a)
    b_len = len(b)
    if a == "": return b_len
    if b == "": return a_len
    
    
    
    # 1. 2차원 표 (a_len+1, b_len+1) 준비하기 
    matrix = [[] for i in range(a_len+1)]
    for i in range(a_len+1): # 0으로 초기화
        matrix[i] = [0 for j in range(b_len+1)]
    
    # 0일 때 초깃값을 설정
    for i in range(a_len+1):
        matrix[i][0] = i
    for j in range(b_len+1):
        matrix[0][j] = j

    
    # 2. 표 채우기
    for i in range(1, a_len+1):
        ac = a[i-1]
        for j in range(1, b_len+1):
            bc = b[j-1]
            cost = 0 if (ac == bc) else 1
            matrix[i][j] = min([
                matrix[i-1][j] + 1,     # 문자 삽입
                matrix[i][j-1] + 1,     # 문자 제거
                matrix[i-1][j-1] + cost # 문자 변경
            ])
    return matrix[a_len][b_len]



# 3. "가나다라"와 "가마바라"의 거리 
print(calc_distance("가나다라","가마바라"))

# 실행 예
samples = ["신촌역","신천군","신천역","신발","마곡역"]
base = samples[0]
r = sorted(samples, key = lambda n: calc_distance(base, n))

for n in r:
    print(calc_distance(base, n), n)

2
0 신촌역
1 신천역
2 신천군
2 신발
2 마곡역


### 2) N-gram으로 유사도 구하기
* N-gram: 텍스트에서 이웃한 N개의 문자를 의미  


이를 활용하면 논문 도용 등을 확인할 수 있다.  
또한 프로그램 코드에 적용해서 라이센스를 가지고 있는 코드를 복사해서 붙여넣지 않았는지 등도 확인할 수 있다.  
검색 엔진, 웹 문서의 유사도를 확인할 수 있다.

In [2]:
def ngram(s, num):
    res = []
    slen = len(s) - num + 1
    for i in range(slen):
        ss = s[i:i+num]
        res.append(ss)
    return res

def diff_ngram(sa, sb, num):
    a = ngram(sa, num)
    b = ngram(sb, num)
    r = []
    cnt = 0
    for i in a:
        for j in b:
            if i == j:
                cnt += 1
                r.append(i)
    return cnt / len(a), r


a = '오늘 강남에서 맛있는 스파게티를 먹었다.'
b = '강남에서 먹었던 오늘의 스파게티는 맛있었다.'

r2, word2 = diff_ngram(a, b, 2)
print('2-gram:', r2, word2)

r3, word3 = diff_ngram(a, b, 3)
print('3-gram:', r3, word3)

2-gram: 0.7619047619047619 ['오늘', '강남', '남에', '에서', '서 ', ' 맛', '맛있', '는 ', ' 스', '스파', '파게', '게티', ' 먹', '먹었', '었다', '다.']
3-gram: 0.45 ['강남에', '남에서', '에서 ', ' 맛있', ' 스파', '스파게', '파게티', ' 먹었', '었다.']
